In [ ]:
import rioxarray as xr
import os
import glob
import pandas as pd
import xarray
import time
import numpy as np
from datetime import datetime

## Extract features of daily mean temperature

In [ ]:
nc_folder_features = "./features/daily_mean_temperature/"
nc_features_paths = glob.glob(nc_folder_features+'*.nc')
nc_features_paths

In [ ]:
for path_name in nc_features_paths:
    features = xarray.open_dataset(path_name)
    features = features['tg']
    features.rio.to_raster(path_name.replace(".nc", ".tif"))

## Extract features of daily precipitation sum

In [ ]:
nc_folder_features = "./features/daily_precipitation_sum/"
nc_features_paths = glob.glob(nc_folder_features+'*.nc')
nc_features_paths

In [ ]:
for path_name in nc_features_paths:
    features = xarray.open_dataset(path_name)
    features = features['rr']
    features.rio.to_raster(path_name.replace(".nc", ".tif"))

## Crop tif files in the 10 regions

In [ ]:
shape_files_dir = "./bacini_shp/"
shape_files = glob.glob(shape_files_dir+'*.shp')
shape_files

In [ ]:
tif_files_dir = "./features/rasters/"
tif_files = glob.glob(tif_files_dir+'*.tif')
tif_files

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import mapping
import geopandas as gpd

for tif_name in tif_files: 
    
    raster = xr.open_rasterio(tif_name)
    
    for shape_name in shape_files:
        
        crop_extent = gpd.read_file(shape_name)
        crop_extent = crop_extent.to_crs(epsg=4326)
        raster = raster.rio.set_crs('epsg:4326')
        tiff_clipped = raster.rio.clip(crop_extent.geometry.apply(mapping), crop_extent.crs)
        tiff_clipped.rio.to_raster(tif_name.replace("_ens_mean_0.1deg_reg", "").replace("_v26.0e.tif", "") + 
                                   shape_name.replace("./bacini_shp/", "_").replace(".shp", "") + '.tif')

### Create csv files from cropped tif files (mean values or all coords) (cyclostationary mean on training set)

In [ ]:
regions = [
    'Adda',
    'Dora',
    'Emiliani1',
    'Piemonte_Sud',
    'Piemonte_Nord',
    'Oglio_Iseo',
    'Ticino',
    'Garda_Mincio',
    'Lambro_Olona',
    'Emiliani2']

tif_files_dir = "./features/rasters/"

# ranges : ['1995-01-01', '2010-12-31'], ['2011-01-01', '2022-06-30']
dates = ['1995-01-01', '2022-06-30']
start = '2000-12-28'
end = '2022-06-24'

days = pd.date_range(start=dates[0], end=dates[1], freq = 'D')
dates_8days = pd.date_range(start=start, end=dates[1], freq = '8D')
#dates_8days_also1weekbf = pd.date_range(start=pd.to_datetime(start)+np.timedelta64(-8,'D'), end=dates[1], freq = '8D')
years = [date.year for date in dates_8days[1:]]
weeks = [date.isocalendar().week for date in dates_8days[1:]]

In [ ]:
def create_statistics_dataframe(complete_dataframe, allCoord):

    n = 0.6
    # take only the first 60% of the dataframe and compute the cyclostationary mean for week
    last_training_day = round(len(dates_8days)*n)
    
    if allCoord == False:
        # create dataframe with mean values for each 8 days and save it as csv
        means = complete_dataframe.mean(axis=0)
        means_8days = [means[i:i+8].mean() for i in range(0, len(means)-1, 8)]
        statistics = pd.DataFrame({'mean': means_8days, 'year': years, 'week': weeks}, index = dates_8days[1:])
        
        train_df = statistics[statistics.index <= dates_8days[last_training_day]]

        weekoftheyar_mean = train_df.groupby(['week'])['mean'].mean()
        index = statistics.index
        statistics = pd.merge(statistics, weekoftheyar_mean, how='left', on=['week'], 
                              suffixes=['','_weekoftheyear']).set_index(index)
        
    else:    
        # create a multi_index with both coordinates and date
        multi_index_dataframe = pd.concat([complete_dataframe.iloc[:,:1]] * len(dates_8days[1:]), 
                                          keys=dates_8days[1:], names=['date'])

        # save mean values for groups of 8 days
        for i in range(0, len(complete_dataframe.columns)-1, 8):
            if i == 0:
                cells_means_8days = complete_dataframe.iloc[:,i:i+8].mean(axis = 1).values
            else:
                cells_means_8days = np.concatenate([cells_means_8days, 
                                                    complete_dataframe.iloc[:,i:i+8].mean(axis = 1).values])

        statistics = pd.DataFrame({'mean': cells_means_8days, 'year': np.repeat(years, len(complete_dataframe)), 
                                   'week': np.repeat(weeks, len(complete_dataframe))}, 
                                  index = multi_index_dataframe.index)

        train_df = statistics[statistics.index.get_level_values(0) <= dates_8days[last_training_day]]
        # cyclostationary_means_8days
        weekoftheyar_mean = train_df.groupby(['week', 'y', 'x'])['mean'].mean()
        index = statistics.index
        statistics = pd.merge(statistics, weekoftheyar_mean, how='left', on=['week', 'y', 'x'], 
                              suffixes=['','_weekoftheyear']).set_index(index)
    
    statistics['cyclostationary_mean'] = statistics['mean'] - statistics['mean_weekoftheyear']
    statistics.drop("mean_weekoftheyear", axis='columns', inplace = True)

    return statistics

In [ ]:
def feature_tifs_to_csv(feature, csv_files_dir, allCoord):
    cropped_tif_files = [[] for i in range(len(dates))]
    for region in regions:
        cropped_tif_files = glob.glob(tif_files_dir + feature + '*' + region + '*.tif')
        cropped_tif_files.sort()

        for i in range(len(cropped_tif_files)):
            raster = xr.open_rasterio(cropped_tif_files[i]).drop_vars(["spatial_ref"])
            dataframe = raster.to_dataset('band').to_dataframe()

            # remove useless null values
            dataframe = dataframe.replace(-9999,np.NaN)
            dataframe = dataframe.dropna()
            
            scale_factor = raster.attrs['scale_factor']
            dataframe = dataframe * scale_factor # fix the scale factor
            
            if i == 0:
                complete_dataframe = dataframe
            else:
                complete_dataframe = pd.concat([complete_dataframe, dataframe], axis=1)

        # convert dates in readable ones and remove useless range
        complete_dataframe.columns = days.strftime('%Y-%m-%d')
        complete_dataframe = complete_dataframe.loc[:, (complete_dataframe.columns >= start) & 
                                                    (complete_dataframe.columns <= end)]

        statistics = create_statistics_dataframe(complete_dataframe, allCoord)
        
        statistics.to_csv(csv_files_dir + region + "_" + feature + ".csv")

In [ ]:
feature_tifs_to_csv('tg', "./features/csv/", False)
feature_tifs_to_csv('rr', "./features/csv/", False)

In [ ]:
feature_tifs_to_csv('tg', "./features/csv_allvalues/", True)
feature_tifs_to_csv('rr', "./features/csv_allvalues/", True)

### Create csv with all temporal aggregations (all coordinates) (cyclostationary mean on training set)

In [ ]:
csv_features_folders = ["./features/csv_allvalues/daily_mean_temperature(tg)/",
                        "./features/csv_allvalues/daily_precipitation_sum(rr)/"]
features = ["tg", "rr"]

In [ ]:
max_aggreg = 24

for region in regions:
    for i, feature in enumerate(features):
        if i != 0:
            # ! there are some duplicates of the year-week pair for special cases, so date considered in join
            df2 = pd.read_csv(csv_features_folders[i] + region + "_" + feature + ".csv")
            df['cyclostationary_mean_' + feature] = df2['cyclostationary_mean']
        else:
            df = pd.read_csv(csv_features_folders[i] + region + "_" + feature + ".csv")
            df.drop('mean', axis = 1, inplace = True)
            df = df.rename(columns={'cyclostationary_mean':'cyclostationary_mean_' + feature})

    # operations and save file
    
    df['date'] = pd.to_datetime(df['date'])

    for column in df.columns[-2:]:
        df[column + '_1w'] = df.apply(lambda x:df.loc[(df['date'] >= x.date+np.timedelta64(-8,'D')) & 
                                      (df['date'] <= x.date) & (df['y'] == x.y) & 
                                      (df['x'] == x.x), column].mean(), axis=1).values
        for i in range(4, max_aggreg+1, 4):
            if i != 20:
                df[column + '_' + str(i) + 'w'] = df.apply(lambda x:df.loc[(df['date']>= x.date+np.timedelta64(-i*8,'D')) & 
                                              (df['date'] <= x.date) & (df['y'] == x.y) & 
                                              (df['x'] == x.x), column].mean(), axis=1).values

    df.set_index('date').to_csv("./features/csv_allvalues/" + region + "_aggreg.csv")

In [ ]:
tg = pd.read_csv("./features/csv_allvalues/temporal_aggreg/Emiliani2_aggreg.csv")

In [ ]:
tg.head(10)

,date,y,x,year,week,cyclostationary_mean_tg,cyclostationary_mean_rr,cyclostationary_mean_tg_1w,cyclostationary_mean_tg_4w,cyclostationary_mean_tg_8w,cyclostationary_mean_tg_12w,cyclostationary_mean_tg_16w,cyclostationary_mean_tg_24w,cyclostationary_mean_rr_1w,cyclostationary_mean_rr_4w,cyclostationary_mean_rr_8w,cyclostationary_mean_rr_12w,cyclostationary_mean_rr_16w,cyclostationary_mean_rr_24w
0,2001-01-05,44.249861,10.44986,2001,1,0.933558,5.559615,0.933558,0.933558,0.933558,0.933558,0.933558,0.933558,5.559615,5.559615,5.559615,5.559615,5.559615,5.559615
1,2001-01-05,44.249861,10.54986,2001,1,0.860962,4.629808,0.860962,0.860962,0.860962,0.860962,0.860962,0.860962,4.629808,4.629808,4.629808,4.629808,4.629808,4.629808
2,2001-01-05,44.349861,10.14986,2001,1,-0.000288,4.400962,-0.000288,-0.000288,-0.000288,-0.000288,-0.000288,-0.000288,4.400962,4.400962,4.400962,4.400962,4.400962,4.400962
3,2001-01-05,44.349861,10.24986,2001,1,-0.220385,4.609615,-0.220385,-0.220385,-0.220385,-0.220385,-0.220385,-0.220385,4.609615,4.609615,4.609615,4.609615,4.609615,4.609615
4,2001-01-05,44.349861,10.34986,2001,1,-0.006538,5.357692,-0.006538,-0.006538,-0.006538,-0.006538,-0.006538,-0.006538,5.357692,5.357692,5.357692,5.357692,5.357692,5.357692
5,2001-01-05,44.349861,10.44986,2001,1,0.303173,5.181731,0.303173,0.303173,0.303173,0.303173,0.303173,0.303173,5.181731,5.181731,5.181731,5.181731,5.181731,5.181731
6,2001-01-05,44.349861,10.54986,2001,1,0.305000,3.524039,0.305000,0.305000,0.305000,0.305000,0.305000,0.305000,3.524039,3.524039,3.524039,3.524039,3.524039,3.524039
7,2001-01-05,44.349861,10.64986,2001,1,0.331635,2.267308,0.331635,0.331635,0.331635,0.331635,0.331635,0.331635,2.267308,2.267308,2.267308,2.267308,2.267308,2.267308
8,2001-01-05,44.349861,10.74986,2001,1,0.382885,2.694231,0.382885,0.382885,0.382885,0.382885,0.382885,0.382885,2.694231,2.694231,2.694231,2.694231,2.694231,2.694231
9,2001-01-05,44.349861,10.84986,2001,1,0.752019,3.956731,0.752019,0.752019,0.752019,0.752019,0.752019,0.752019,3.956731,3.956731,3.956731,3.956731,3.956731,3.956731
